# How To: Adding Hunting Bookmarks from Notebooks

__Notebook Version:__ 1.0<br>
__Python Version:__ Python 3.6 (including Python 3.6 - AzureML)<br>
__Required Packages:__ azure 4.0.0,  azure-cli-profile 2.1.4<br>
__Platforms Supported:__<br>
    -  Azure Notebooks Free Compute
    -  Azure Notebooks DSVM
__Data Source Required:__<br>
    -  no
    
### Description
The sample notebook shows how to add hunting bookmarks from Jupyter notebooks to Sentinel portal.

In [ ]:
!pip install Azure-Sentinel-Utilities  --upgrade
!pip install jsons

In [ ]:
from SentinelPortal import Constants, BookmarkProperties, BookmarkModel, BookmarkHelper
from azure.common.credentials import get_azure_cli_credentials

In [ ]:
# please enter your tenant domain below, for Microsoft, using: microsoft.onmicrosoft.com
!az login --tenant 'microsoft.onmicrosoft.com'

In [ ]:
# Retrieve access token
creds, sub_id = get_azure_cli_credentials()
token = creds._token_retriever()
access_token = token[2]['accessToken']

In [ ]:
# User Input
subscription_id = input('Subscription Id: ')
resource_group_name = input('Resource Group: ')
workspace_name = input('LA Workspace Name: ')

In [ ]:
entity_mappings = {}

In [ ]:
entity_mappings.update({"550a6d02-d667-49d8-969a-e709cce03293": "Account"})
entity_mappings.update({"201.12.34.111": "Host"})

In [ ]:
# Construct properties
properties = BookmarkProperties("New Test 001", query="AzureActivity | take 1", query_result_dict=entity_mappings, tag_list=None, notes=None, event_time=None, query_start_time=None, query_end_time=None)
#properties = BookmarkProperties("New nb test  10021621", query="AzureActivity | take 1", query_result="{\"__entityMapping\":{\"550a6d02-d667-49d8-969a-e709cce03293\":\"Account\", \"201.12.34.88\":\"Host\"}}", tag_list=None, notes=None, event_time=None, query_start_time=None, query_end_time=None)

In [ ]:
model = BookmarkModel("New Test 001", subscription_id, resource_group_name, workspace_name, properties)

In [ ]:
helper = BookmarkHelper(access_token)

In [ ]:
result = helper.get_bookmarks(model)

In [ ]:
print(result.text)

In [ ]:
result = helper.add_bookmark(model)

In [ ]:
print(result.text)